In [1]:
import numpy as np
from tqdm import tqdm
from echo_lv.data import LV_CAMUS_Dataset, LV_EKB_Dataset
from echo_lv.metrics import dice as dice_np
import torch
import torchvision
from torch.utils.data import DataLoader
from torch import sigmoid
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from torchsummary import summary
from echo_lv.utils import AverageMeter
import segmentation_models_pytorch as smp
from com_unet import UNet
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

random_state = 17
torch.manual_seed(random_state)
torch.cuda.manual_seed(random_state)
torch.backends.cudnn.deterministic = True

cuda:0


In [3]:
batch = 4
epochs = 60
folds = None

lv_camus = LV_CAMUS_Dataset(img_size = (388,388), classes = {0, 1}, folds=folds)
train_loader = DataLoader(lv_camus, batch_size=batch, shuffle=True, num_workers=2)

weight = 10 * torch.ones((1,1,388,388), device=device).to(device)
criterion = smp.utils.losses.BCEWithLogitsLoss(pos_weight=weight).to(device)
# criterion = smp.utils.losses.DiceLoss(activation='sigmoid')# + smp.utils.losses.BCEWithLogitsLoss(pos_weight=weight) 
dice = smp.utils.metrics.Fscore(activation='sigmoid', threshold=None).to(device)#Dice()
iou = smp.utils.metrics.IoU(activation='sigmoid', threshold=None).to(device)


header = True

model = UNet(n_channels = 1, n_classes = 1, bilinear=False).to(device)

optimizer = torch.optim.SGD([
        {'params': model.parameters(), 'lr': 1e-4, 'momentum' : 0.99},   
    ])

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

t = tqdm(total=epochs, 
        bar_format='{desc} | {postfix[0]}/'+ str(epochs) +' | ' +
        '{postfix[1]} : {postfix[2]:>2.4f} | {postfix[3]} : {postfix[4]:>2.4f} | {postfix[5]} : {postfix[6]:>2.4f} |',
        postfix=[0, 'loss', 0, 'dice_lv', 0,  'jaccard_lv', 0,], 
        desc = 'Train common unet ',
        position=0, leave=True
     )            


for epoch in range(0, epochs):
    average_total_loss = AverageMeter()
    average_dice = AverageMeter()
    average_jaccard = AverageMeter()

    model.train()

    t.postfix[0] = epoch + 1

    for data in train_loader:

        inputs, masks, *_ = data
        shape = inputs.shape
        inputs = torch.cat([torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float), inputs, torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float)], axis=3)
        shape = inputs.shape
        inputs = torch.cat([torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float), inputs, torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float)], axis=2)
        inputs=inputs.to(device).float()
        masks=masks.to(device).float()

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, masks)

        average_total_loss.update(loss.data.item())
        average_dice.update(dice(outputs, masks).item())
        average_jaccard.update(iou(outputs, masks).item())

        loss.backward()
        optimizer.step()

        t.postfix[2] = average_total_loss.average()
        t.postfix[4] = average_dice.average()
        t.postfix[6] = average_jaccard.average()
        t.update(n=1)

    result = [optimizer.param_groups[0]['lr'], 
              average_total_loss.average(), 
              average_dice.average(), 
              average_jaccard.average(),
             ]
    df = pd.DataFrame(np.array([result]), columns=['lr', 'loss', 'dice', 'jaccard'])
    df.to_csv('cnn/com_unet/result.csv', mode='a', header=header, index=False,)
    header=None
    scheduler.step()
t.close()

Train common unet  | 21/60 | loss : 0.0695 | dice_lv : 0.8794 | jaccard_lv : 0.7866 |IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Train common unet  | 38/60 | loss : 0.0417 | dice_lv : 0.9304 | jaccard_lv : 0.8704 |IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Train common unet  | 60/60 | loss : 0.0224 | dice_lv : 0.9619 | jaccard_lv : 0.9267 |


In [4]:
torch.save(model.to('cpu').state_dict(), 'common_unet.pth')

In [2]:
batch = 4
epochs = 10
folds = None

lv_ekb = LV_EKB_Dataset(img_size = (388,388), normalize=True, only_first_frames=True)
train_loader = DataLoader(lv_ekb, batch_size=batch, shuffle=True, num_workers=2)

weight = 10 * torch.ones((1,1,388,388), device=device).to(device)
criterion = smp.utils.losses.BCEWithLogitsLoss(pos_weight=weight).to(device)
# criterion = smp.utils.losses.DiceLoss(activation='sigmoid')# + smp.utils.losses.BCEWithLogitsLoss(pos_weight=weight) 
dice = smp.utils.metrics.Fscore(activation='sigmoid', threshold=None).to(device)#Dice()
iou = smp.utils.metrics.IoU(activation='sigmoid', threshold=None).to(device)


header = True

model = UNet(n_channels = 1, n_classes = 1, bilinear=False).to(device)
model.load_state_dict(torch.load('common_unet.pth'))

optimizer = torch.optim.SGD([
        {'params': model.parameters(), 'lr': 1e-4, 'momentum' : 0.99},   
    ])

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

t = tqdm(total=epochs, 
        bar_format='{desc} | {postfix[0]}/'+ str(epochs) +' | ' +
        '{postfix[1]} : {postfix[2]:>2.4f} | {postfix[3]} : {postfix[4]:>2.4f} | {postfix[5]} : {postfix[6]:>2.4f} |',
        postfix=[0, 'loss', 0, 'dice_lv', 0,  'jaccard_lv', 0,], 
        desc = 'Train common unet ',
        position=0, leave=True
     )            


for epoch in range(0, epochs):
    average_total_loss = AverageMeter()
    average_dice = AverageMeter()
    average_jaccard = AverageMeter()

    model.train()

    t.postfix[0] = epoch + 1

    for data in train_loader:

        inputs, masks, *_ = data
        shape = inputs.shape
        inputs = torch.cat([torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float), inputs, torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float)], axis=3)
        shape = inputs.shape
        inputs = torch.cat([torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float), inputs, torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float)], axis=2)
        inputs=inputs.to(device).float()
        masks=masks.to(device).float()

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, masks)

        average_total_loss.update(loss.data.item())
        average_dice.update(dice(outputs, masks).item())
        average_jaccard.update(iou(outputs, masks).item())

        loss.backward()
        optimizer.step()

        t.postfix[2] = average_total_loss.average()
        t.postfix[4] = average_dice.average()
        t.postfix[6] = average_jaccard.average()
        t.update(n=1)

    result = [optimizer.param_groups[0]['lr'], 
              average_total_loss.average(), 
              average_dice.average(), 
              average_jaccard.average(),
             ]
    df = pd.DataFrame(np.array([result]), columns=['lr', 'loss', 'dice', 'jaccard'])
#     df.to_csv('cnn/com_unet/result.csv', mode='a', header=header, index=False,)
    header=None
    scheduler.step()
t.close()

Dataset is correct.


Train common unet  | 10/10 | loss : 0.0359 | dice_lv : 0.8763 | jaccard_lv : 0.7804 |


In [3]:
torch.save(model.to('cpu').state_dict(), 'common_unet_1st_frame.pth')

In [3]:
summary(model, (1, 572, 572), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 570, 570]             640
              ReLU-2         [-1, 64, 570, 570]               0
            Conv2d-3         [-1, 64, 568, 568]          36,928
              ReLU-4         [-1, 64, 568, 568]               0
        DoubleConv-5         [-1, 64, 568, 568]               0
         MaxPool2d-6         [-1, 64, 284, 284]               0
            Conv2d-7        [-1, 128, 282, 282]          73,856
              ReLU-8        [-1, 128, 282, 282]               0
            Conv2d-9        [-1, 128, 280, 280]         147,584
             ReLU-10        [-1, 128, 280, 280]               0
       DoubleConv-11        [-1, 128, 280, 280]               0
             Down-12        [-1, 128, 280, 280]               0
        MaxPool2d-13        [-1, 128, 140, 140]               0
           Conv2d-14        [-1, 256, 1

In [4]:
folds = 10
lv_camus = LV_CAMUS_Dataset(img_size = (388,388), classes = {0, 1}, folds=folds)
# lv_camus_valid = LV_CAMUS_Dataset(img_size = (572,572), classes = {0, 1}, folds=10, subset='valid')
lv_camus.set_state('train', 0)
train_loader = DataLoader(lv_camus, batch_size=1, shuffle=True, num_workers=4)

In [7]:
lv_camus.set_state(subset='valid', fold=9)
len(lv_camus)

144

In [8]:
lv_camus.set_state(subset='train', fold=9)
len(lv_camus)

1656

In [ ]:
batch = 4
epochs = 20
folds = 9

lv_camus = LV_CAMUS_Dataset(img_size = (388,388), classes = {0, 1}, folds=folds)

weight = 10 * torch.ones((1,1,388,388), device=device)
criterion = smp.utils.losses.BCEWithLogitsLoss(pos_weight=weight)
# criterion = smp.utils.losses.DiceLoss(activation='sigmoid')# + smp.utils.losses.BCEWithLogitsLoss(pos_weight=weight) 
dice = smp.utils.metrics.Fscore(activation='sigmoid', threshold=0.5)#Dice()
iou = smp.utils.metrics.IoU(activation='sigmoid', threshold=0.5)


for fold in range(1,2):
    model = UNet(n_channels = 1, n_classes = 1, bilinear=False).to(device)
    
    optimizer = torch.optim.SGD([
            {'params': model.parameters(), 'lr': 1e-3},   
        ])
    
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    
    t = tqdm(total=epochs, 
            bar_format='{desc} | {postfix[0]}/'+ str(epochs) +' | ' +
            '{postfix[1]} : {postfix[2]:>2.4f} | {postfix[3]} : {postfix[4]:>2.4f} | {postfix[5]} : {postfix[6]:>2.4f} |' +
            '{postfix[7]} : {postfix[8]:>2.4f} | {postfix[9]} : {postfix[10]:>2.4f} | {postfix[11]} : {postfix[12]:>2.4f} |',
            postfix=[0, 'loss', 0, 'dice_lv', 0,  'jaccard_lv', 0,
                     'val_loss', 0, 'val_dice_lv', 0, 'val_jaccard_lv', 0], 
            desc = 'Train common unet on fold ' + str(fold),
            position=0, leave=True
         )            
        
       
    for epoch in range(0, epochs):
        average_total_loss = AverageMeter()
        average_dice = AverageMeter()
        average_jaccard = AverageMeter()
        
        torch.cuda.empty_cache()
        model.train()

        t.postfix[0] = epoch + 1
        
        lv_camus.set_state('train', fold)
        train_loader = DataLoader(lv_camus, batch_size=batch, shuffle=True, num_workers=4)
        for data in train_loader:

            torch.cuda.empty_cache()
            inputs, masks, *_ = data
            shape = inputs.shape
            inputs = torch.cat([torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float), inputs, torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float)], axis=3)
            shape = inputs.shape
            inputs = torch.cat([torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float), inputs, torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float)], axis=2)
            inputs=inputs.to(device).float()
            masks=masks.to(device).float()

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, masks)

            average_total_loss.update(loss.data.item())
            average_dice.update(dice(outputs, masks).item())
            average_jaccard.update(iou(outputs, masks).item())

            loss.backward()
            optimizer.step()

            t.postfix[2] = average_total_loss.average()
            t.postfix[4] = average_dice.average()
            t.postfix[6] = average_jaccard.average()
            t.update(n=1)

        # validation
        average_val_total_loss = AverageMeter()
        average_val_dice = AverageMeter()
        average_val_jaccard = AverageMeter()
        model.eval()
        
        lv_camus.set_state('valid', fold)
        valid_loader = DataLoader(lv_camus, batch_size=batch, shuffle=True, num_workers=2)
        for data in valid_loader:
            inputs, masks, *_ = data
            shape = inputs.shape
            inputs = torch.cat([torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float), inputs, torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float)], axis=3)
            shape = inputs.shape
            inputs = torch.cat([torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float), inputs, torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float)], axis=2)
            inputs=inputs.to(device).float()
            masks=masks.to(device).float()

            outputs = model(inputs)

            loss = criterion(outputs, masks)

            average_val_total_loss.update(loss.data.item())
            average_val_dice.update(dice(outputs, masks).item())
            average_val_jaccard.update(iou(outputs, masks).item())

            t.postfix[8] = average_val_total_loss.average()
            t.postfix[10] = average_val_dice.average()
            t.postfix[12] = average_val_jaccard.average()
            t.update(n=0)
        
        scheduling.step()
        
        result = [average_total_loss.average(), average_dice.average(), average_jaccard.average(),
                  average_val_total_loss.average(), average_val_dice.average(), average_val_jaccard.average()
                 ]
        df = pd.DataFrame(np.array(result), columns=['loss_' + str(fold), 'dice_' + str(fold), 'jaccard_' + str(fold), 
                                                     'val_loss_' + str(fold), 'val_dice_' + str(fold), 'val_jaccard_' + str(fold)])
        df.to_csv('cnn/com_unet/result_cunet_'+ str(fold) +'.csv', mode='a', header=header, index=False,)
        header=None
    t.close()

Train common unet on fold 1 | 1/20 | loss : 0.8268 | dice_lv : 0.2422 | jaccard_lv : 0.1407 |val_loss : 0.0000 | val_dice_lv : 0.0000 | val_jaccard_lv : 0.0000 |

In [10]:
torch.save(model.state_dict(), 'common_unet_wo_bn.pth')

In [ ]:
batch = 4
epochs = 20
folds = 9

lv_camus = LV_CAMUS_Dataset(img_size = (388,388), classes = {0, 1}, folds=folds)

weight = 10 * torch.ones((1,1,388,388), device=device)
criterion = smp.utils.losses.BCEWithLogitsLoss(pos_weight=weight)
# criterion = smp.utils.losses.DiceLoss(activation='sigmoid')# + smp.utils.losses.BCEWithLogitsLoss(pos_weight=weight) 
dice = smp.utils.metrics.Fscore(activation='sigmoid', threshold=0.5)#Dice()
iou = smp.utils.metrics.IoU(activation='sigmoid', threshold=0.5)


for fold in range(1,2):
    model = UNet(n_channels = 1, n_classes = 1, bilinear=False).to(device)
    
    optimizer = torch.optim.SGD([
            {'params': model.parameters(), 'lr': 1e-3},   
        ])
    
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    
    t = tqdm(total=epochs, 
            bar_format='{desc} | {postfix[0]}/'+ str(epochs) +' | ' +
            '{postfix[1]} : {postfix[2]:>2.4f} | {postfix[3]} : {postfix[4]:>2.4f} | {postfix[5]} : {postfix[6]:>2.4f} |' +
            '{postfix[7]} : {postfix[8]:>2.4f} | {postfix[9]} : {postfix[10]:>2.4f} | {postfix[11]} : {postfix[12]:>2.4f} |',
            postfix=[0, 'loss', 0, 'dice_lv', 0,  'jaccard_lv', 0,
                     'val_loss', 0, 'val_dice_lv', 0, 'val_jaccard_lv', 0], 
            desc = 'Train common unet on fold ' + str(fold),
            position=0, leave=True
         )            
        
       
    for epoch in range(0, epochs):
        average_total_loss = AverageMeter()
        average_dice = AverageMeter()
        average_jaccard = AverageMeter()
        
        torch.cuda.empty_cache()
        model.train()

        t.postfix[0] = epoch + 1
        
        lv_camus.set_state('train', fold)
        train_loader = DataLoader(lv_camus, batch_size=batch, shuffle=True, num_workers=4)
        for data in train_loader:

            torch.cuda.empty_cache()
            inputs, masks, *_ = data
            shape = inputs.shape
            inputs = torch.cat([torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float), inputs, torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float)], axis=3)
            shape = inputs.shape
            inputs = torch.cat([torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float), inputs, torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float)], axis=2)
            inputs=inputs.to(device).float()
            masks=masks.to(device).float()

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, masks)

            average_total_loss.update(loss.data.item())
            average_dice.update(dice(outputs, masks).item())
            average_jaccard.update(iou(outputs, masks).item())

            loss.backward()
            optimizer.step()

            t.postfix[2] = average_total_loss.average()
            t.postfix[4] = average_dice.average()
            t.postfix[6] = average_jaccard.average()
            t.update(n=1)

        # validation
        average_val_total_loss = AverageMeter()
        average_val_dice = AverageMeter()
        average_val_jaccard = AverageMeter()
        model.eval()
        
        lv_camus.set_state('valid', fold)
        valid_loader = DataLoader(lv_camus, batch_size=batch, shuffle=True, num_workers=2)
        for data in valid_loader:
            inputs, masks, *_ = data
            shape = inputs.shape
            inputs = torch.cat([torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float), inputs, torch.zeros((shape[0], shape[1], shape[2], 92), dtype=float)], axis=3)
            shape = inputs.shape
            inputs = torch.cat([torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float), inputs, torch.zeros((shape[0], shape[1], 92, shape[3]), dtype=float)], axis=2)
            inputs=inputs.to(device).float()
            masks=masks.to(device).float()

            outputs = model(inputs)

            loss = criterion(outputs, masks)

            average_val_total_loss.update(loss.data.item())
            average_val_dice.update(dice(outputs, masks).item())
            average_val_jaccard.update(iou(outputs, masks).item())

            t.postfix[8] = average_val_total_loss.average()
            t.postfix[10] = average_val_dice.average()
            t.postfix[12] = average_val_jaccard.average()
            t.update(n=0)
        
        scheduling.step()
        
        result = [average_total_loss.average(), average_dice.average(), average_jaccard.average(),
                  average_val_total_loss.average(), average_val_dice.average(), average_val_jaccard.average()
                 ]
        df = pd.DataFrame(np.array(result), columns=['loss_' + str(fold), 'dice_' + str(fold), 'jaccard_' + str(fold), 
                                                     'val_loss_' + str(fold), 'val_dice_' + str(fold), 'val_jaccard_' + str(fold)])
        df.to_csv('cnn/com_unet/result_cunet_'+ str(fold) +'.csv', mode='a', header=header, index=False,)
        header=None
    t.close()